In [1]:
from openai import OpenAI

client = OpenAI(
    api_key="EMPTY",
    base_url="http://localhost:8000/v1"
)

In [ ]:
text = ""
prompt = f"""Ниже — текст, полученный распознаванием аудио. В нём могут встречаться слова, которые не вписываются в смысл фразы, потому что были неправильно распознаны. Такие слова нужно заменить на фонетически похожие, которые логически подходят в контексте.

ВАЖНО:
- Не переписывай и не улучшай текст целиком, не добавляй и не убирай слова без необходимости.
- Не меняй стиль речи, даже если он разговорный или неформальный.
- Исправляй только те слова, которые явно выбиваются из смысла — замени их на звучащие похоже, если знаешь, что имелось в виду.
- Если сомневаешься — оставь как есть.
- Не возвращай ничего кроме отредактированного текста.

Текст: {text}

Отредактированный текст:
"""
resp = client.chat.completions.create(
    model="Qwen/Qwen3-4B",
    messages=[
      {"role": "system", "content": "Ты — помощник по корректировке автоматической транскрипции речи."},
      {"role": "user", "content": prompt}
    ],
    temperature=0.7,
    top_p=0.9,
    extra_body={"top_k":20},
    max_completion_tokens=32000
)

print(resp.choices[0].message.content)

In [2]:
import os
import json
from tqdm import tqdm

def generate_prompt(prev_text, current_text, use_context=True):
    if use_context and prev_text.strip():
        return f"""Ниже приведены две реплики диалога: предыдущая и текущая. Текущая реплика получена распознаванием аудио. В ней могут встречаться слова, которые не вписываются в смысл фразы, потому что были неправильно распознаны. Такие слова нужно заменить на фонетически похожие, которые логически подходят в контексте.

Твоя задача:
- Исправить только те слова в текущей реплике, которые явно не вписываются в контекст и, скорее всего, были ошибкой распознавания.
- Подбирать замену только из слов, которые звучат похоже (фонетически), и лучше подходят по смыслу.
- Не переписывать реплику целиком. Не менять стиль, не переформулировать.
- Не трогать нормально звучащие и логичные фразы — даже если они разговорные.
- Не добавлять и не убирать слова без необходимости.
- Если сомневаешься — оставь как есть.
- Не возвращай ничего, кроме отредактированного текста.

Контекст — предыдущая реплика собеседника:
{prev_text}

Текущая реплика (проверь и исправь при необходимости):
{current_text}

Исправленный вариант текущей реплики:
"""
    else:
        return f"""Ниже — текст, полученный распознаванием аудио. В нём могут встречаться слова, которые не вписываются в смысл фразы, потому что были неправильно распознаны. Такие слова нужно заменить на фонетически похожие, которые логически подходят в контексте.

ВАЖНО:
- Не переписывай и не улучшай текст целиком.
- Не меняй стиль речи, даже если он разговорный или неформальный.
- Не добавляй и не убирай слова без необходимости.
- Подбирать замену только из слов, которые звучат похоже (фонетически), и лучше подходят по смыслу.
- Если сомневаешься — оставь как есть.
- Не возвращай ничего, кроме отредактированного текста.

Текст: {current_text}

Отредактированный текст:
"""

In [ ]:
import os
import json
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

def call_api_for_entry(system_msg, user_prompt):
    try:
        resp = client.chat.completions.create(
            model="Qwen/Qwen3-4B",
            messages=[
                {"role": "system", "content": system_msg},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.7,
            top_p=0.9,
            extra_body={"top_k": 20},
            max_completion_tokens=32000
        )
        return resp.choices[0].message.content.strip()
    except Exception as e:
        print(f"Ошибка API: {e}")
        return None

def process_transcript_file_parallel(filepath, output_dir, max_workers=5):
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)

    processed = [None] * len(data)
    system_msg = "Ты — помощник по корректировке автоматической транскрипции речи."

    prompts = []
    prev_text = ""
    for idx, entry in enumerate(data):
        use_context = idx > 0
        prompt = generate_prompt(prev_text, entry["text"], use_context=use_context)
        prompts.append(prompt)
        prev_text = entry["text"]

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {}
        for idx, prompt in enumerate(prompts):
            futures[executor.submit(call_api_for_entry, system_msg, prompt)] = idx

        for future in tqdm(as_completed(futures), total=len(futures), desc=f"Обработка {os.path.basename(filepath)}"):
            idx = futures[future]
            result = future.result()
            if result is None:
                result = data[idx]["text"]
            data[idx]["corrected_text"] = result
            processed[idx] = data[idx]

    os.makedirs(output_dir, exist_ok=True)
    out_path = os.path.join(output_dir, os.path.basename(filepath))
    with open(out_path, 'w', encoding='utf-8') as f:
        json.dump(processed, f, ensure_ascii=False, indent=2)

input_folder = "transcripts"
output_folder = "corrected_transcripts"

for filename in os.listdir(input_folder):
    if filename.endswith(".json"):
        print(f"\nОбработка файла {filename}")
        process_transcript_file_parallel(os.path.join(input_folder, filename), output_folder, max_workers=5)


Обработка файла А2_3_formatted.json


Обработка А2_3_formatted.json:  35%|███▍      | 165/472 [31:02<6:45:33, 79.26s/it]

Ошибка API: Request timed out.


Обработка А2_3_formatted.json:  48%|████▊     | 226/472 [53:19<1:44:34, 25.51s/it]

Ошибка API: Request timed out.


...

Обработка A1_6(1)_formatted.json:  65%|██████▌   | 79/121 [32:12<32:37, 46.61s/it]

Ошибка API: Request timed out.


Обработка A1_6(1)_formatted.json:  98%|█████████▊| 119/121 [54:46<01:57, 58.63s/it]

Ошибка API: Request timed out.


Обработка A1_6(1)_formatted.json: 100%|██████████| 121/121 [1:09:26<00:00, 34.44s/it] 
